In [1]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='"Heeeey~ You know, it\'s pretty amusing watching all of you work so hard to catch up. But that\'s what makes it interesting! Being the strongest isn\'t just about raw power - though I\'ve got plenty of that adjusts blindfold with a grin\nYou see, when you have the Six Eyes and Limitless like I do, you start to see things differently. Literally! The world is an endless flow of cursed energy, beautiful and dangerous all at once. But power isn\'t everything. Well... maybe it is laughs Just kidding!\nThe real strength comes from understanding what you\'re fighting for. Me? I fight to change this backwards jujutsu world, to protect my students, and honestly... because it\'s fun! There\'s nothing quite like showing a Special Grade curse who\'s boss.\nTo all my adorable students - yes, even you, Yuuji - keep growing stronger. Though you\'ll never be as strong as me winks But that\'s fine! Not everyone can be the honored one.\nAnd to a

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")



In [13]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup, SoupStrainer  # Import BeautifulSoup if needed elsewhere

# Define the loader for the webpage
loader = WebBaseLoader(
    web_paths=["https://uselessai.in/introduction-to-machine-learning-e272cf75b5b0"]  # Correct the input to a list
)

# Load the documents
documents = loader.load()

In [ ]:
##PDF
## loader = PyPDFLoader()

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)

# Ensure documents are valid and not empty
valid_documents = [doc for doc in documents if hasattr(doc, "page_content") and doc.page_content]

# Split text into chunks
chunks = []
for doc in valid_documents:
    chunks.extend(text_splitter.split_text(doc.page_content))

# Debugging output
print(f"Number of chunks created: {len(chunks)}")


Number of chunks created: 10


In [83]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from uuid import uuid4

embeddings = OllamaEmbeddings(model="llama3")
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)
  # Where to save data locally, remove if not necessary
uuids = [str(uuid4()) for _ in range(len(chunks))]
vector_store.add_documents(documents=documents, ids=uuids)

['b6b412b5-c4c1-4e49-8b82-e573a24fb91e',
 'a43f8ec2-1abe-4b10-aaaa-779c2d12f5d8',
 '7d9e2fef-71b7-4033-a249-2d812824f19b',
 '03863113-8f32-42af-aff0-eec7db7b8f98',
 'e0e06ff5-bffa-4c75-adfb-1d46f84c4730',
 '4ce11ca8-6090-430b-a230-cafa99a77893',
 'f2fb6487-3a1b-415d-9e19-51597450f026',
 'b8b5e50f-7e47-4fde-a195-2e7d10562132',
 '5fd2c33e-fc0a-4cd3-b4a4-673cb117232c']

In [85]:
query="who to start with machine learnig"
result= vector_store.similarity_search(query)
result

[Document(id='8c36ba4f-054d-4a71-b097-dd523ad091bd', metadata={}, page_content='The sixth chunk of text.'),
 Document(id='4ce11ca8-6090-430b-a230-cafa99a77893', metadata={}, page_content='The sixth chunk of text.'),
 Document(id='38ec4c79-bba5-45a4-9c45-09d8f939aabd', metadata={}, page_content='Another example chunk of text.'),
 Document(id='03863113-8f32-42af-aff0-eec7db7b8f98', metadata={}, page_content='Another example chunk of text.')]